In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# ExecComp


`ExecComp` is a component that provides a shortcut for building an ExplicitComponent that
represents a set of simple mathematical relationships between inputs and outputs. The ExecComp
automatically takes care of all of the component API methods, so you just need to instantiate
it with an equation or a list of equations.

## ExecComp Options


In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.components.exec_comp.ExecComp")

## ExecComp Constructor

The call signature for the `ExecComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.exec_comp.ExecComp.__init__
        :noindex:
```

The values of the `kwargs` can be `dicts` which define the initial value for the variables along with
other metadata. For example,

```
    ExecComp('xdot=x/t', x={'units': 'ft'}, t={'units': 's'}, xdot={'units': 'ft/s')
```

Here is a list of the possible metadata that can be assigned to a variable in this way. The **Applies To** column indicates
whether the metadata is appropriate for input variables, output variables, or both.

```{eval-rst}
================  ====================================================== ============================================================= ==============  ========
Name              Description                                            Valid Types                                                   Applies To      Default
================  ====================================================== ============================================================= ==============  ========
value             Initial value in user-defined units                    float, list, tuple, ndarray                                   input & output  1
shape             Variable shape, only needed if not an array            int, tuple, list, None                                        input & output  None
shape_by_conn     Determine variable shape based on its connection       bool                                                          input & output  False
copy_shape        Determine variable shape based on named variable       str                                                           input & output  None
units             Units of variable                                      str, None                                                     input & output  None
desc              Description of variable                                str                                                           input & output  ""
res_units         Units of residuals                                     str, None                                                     output          None
ref               Value of variable when scaled value is 1               float, ndarray                                                output          1
ref0              Value of variable when scaled value is 0               float, ndarray                                                output          0
res_ref           Value of residual when scaled value is 1               float, ndarray                                                output          1
lower             Lower bound of variable                                float, list, tuple, ndarray, Iterable, None                   output          None
upper             Lower bound of variable                                float, list, tuple, ndarray, Iterable, None                   output          None
src_indices       Global indices of the variable                         int, list of ints, tuple of ints, int ndarray, Iterable, None input           None
flat_src_indices  If True, src_indices are indices into flattened source bool                                                          input           None
tags              Used to tag variables for later filtering              str, list of strs                                             input & output  None
================  ====================================================== ============================================================= ==============  ========
```

These metadata are passed to the `Component` methods `add_input` and `add_output`.
For more information about these metadata, see the documentation for the arguments to these methods on [Component](../../../_srcdocs/packages/core/component).

## Registering User Functions

To get your own functions added to the internal namespace of ExecComp so you can call them
from within an ExecComp expression, you can use the `ExecComp.register` function.

```{eval-rst}
    .. automethod:: openmdao.components.exec_comp.ExecComp.register
        :noindex:
```

Note that you're required, when registering a new function, to indicate whether that function
is complex safe or not.


## ExecComp Example: Simple

For example, here is a simple component that takes the input and adds one to it.

In [ ]:
import openmdao.api as om

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', om.ExecComp('y=x+1.'))

model.set_input_defaults('comp.x', 2.0)

prob.setup()

prob.run_model()

print(prob.get_val('comp.y'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(prob.get_val('comp.y'), 3.0, 0.00001)

## ExecComp Example: Multiple Outputs

You can also create an ExecComp with multiple outputs by placing the expressions in a list.

In [ ]:
prob = om.Problem()
model = prob.model

model.add_subsystem('comp', om.ExecComp(['y1=x+1.', 'y2=x-1.']), promotes=['x'])

prob.setup()

prob.set_val('x', 2.0)

prob.run_model()

print(prob.get_val('comp.y1'))
print(prob.get_val('comp.y2'))

In [ ]:
assert_near_equal(prob.get_val('comp.y1'), 3.0, 0.00001)
assert_near_equal(prob.get_val('comp.y2'), 1.0, 0.00001)

## ExecComp Example: Arrays

You can declare an ExecComp with arrays for inputs and outputs, but when you do, you must also
pass in a correctly-sized array as an argument to the ExecComp call, or set the 'shape' metadata
for that variable as described earlier. If specifying the value directly, it can be the initial value
in the case of unconnected inputs, or just an empty array with the correct size.

In [ ]:
import numpy as np

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', om.ExecComp('y=x[1]',
                                        x=np.array([1., 2., 3.]),
                                        y=0.0))

prob.setup()

prob.run_model()

print(prob.get_val('comp.y'))

In [ ]:
assert_near_equal(prob.get_val('comp.y'), 2.0, 0.00001)

## ExecComp Example: Math Functions

Functions from the math library are available for use in the expression strings.

In [ ]:
import numpy as np

prob = om.Problem()
model = prob.model

model.add_subsystem('comp', om.ExecComp('z = sin(x)**2 + cos(y)**2'))

prob.setup()

prob.set_val('comp.x', np.pi/2.0)
prob.set_val('comp.y', np.pi/2.0)

prob.run_model()

print(prob.get_val('comp.z'))

In [ ]:
assert_near_equal(prob.get_val('comp.z'), 1.0, 0.00001)

## ExecComp Example: Variable Properties

You can also declare properties like 'units', 'upper', or 'lower' on the inputs and outputs. In this
example we declare all our inputs to be inches to trigger conversion from a variable expressed in feet
in one connection source.

In [ ]:
prob = om.Problem()
model = prob.model

model.add_subsystem('comp', om.ExecComp('z=x+y',
                                        x={'val': 0.0, 'units': 'inch'},
                                        y={'val': 0.0, 'units': 'inch'},
                                        z={'val': 0.0, 'units': 'inch'}))

prob.setup()

prob.set_val('comp.x', 12.0, units='inch')
prob.set_val('comp.y', 1.0, units='ft')

prob.run_model()

print(prob.get_val('comp.z'))

In [ ]:
assert_near_equal(prob.get_val('comp.z'), 24.0, 0.00001)

## ExecComp Example: Diagonal Partials

If all of your ExecComp's array inputs and array outputs are the same size and happen to have
diagonal partials, you can make computation of derivatives for your ExecComp faster by specifying a
`has_diag_partials=True` argument
to `__init__` or via the component options. This will cause the ExecComp to solve for its partials
by complex stepping all entries of an array input at once instead of looping over each entry individually.

In [ ]:
import numpy as np

p = om.Problem()
model = p.model

model.add_subsystem('comp', om.ExecComp('y=3.0*x + 2.5',
                                        has_diag_partials=True,
                                        x=np.ones(5), y=np.ones(5)))

p.setup()

p.set_val('comp.x', np.ones(5))

p.run_model()

J = p.compute_totals(of=['comp.y'], wrt=['comp.x'], return_format='array')

print(J)

In [ ]:
from numpy.testing import assert_almost_equal

assert_almost_equal(J, np.eye(5)*3., decimal=6)

## ExecComp Example: Options

Other options that can apply to all the variables in the component are variable shape and units.
These can also be set as a keyword argument in the constructor or via the component options. In the
following example the variables all share the same shape, which is specified in the constructor, and
common units that are specified by setting the option.

In [ ]:
model = om.Group()

xcomp = model.add_subsystem('comp', om.ExecComp('y=2*x', shape=(2,)))

xcomp.options['units'] = 'm'

prob = om.Problem(model)
prob.setup()

prob.set_val('comp.x', [100., 200.], units='cm')

prob.run_model()

print(prob.get_val('comp.y'))

In [ ]:
assert_near_equal(prob.get_val('comp.y'), [2., 4.], 0.00001)

## ExecComp Example: User function registration

If the function is complex safe, then you don't need to do anything differently than you
would for any other ExecComp.

In [ ]:
try: 
    om.ExecComp.register("myfunc", lambda x: x * x, complex_safe=True)
except NameError:
    pass
p = om.Problem()
comp = p.model.add_subsystem("comp", om.ExecComp("y = 2 * myfunc(x)"))

p.setup()
p.run_model()
J = p.compute_totals(of=['comp.y'], wrt=['comp.x'])
print(J['comp.y', 'comp.x'][0][0])

In [ ]:
assert_near_equal(J['comp.y', 'comp.x'][0][0], 4., 1e-10)

## ExecComp Example: Complex unsafe user function registration

If the function isn't complex safe, then derivatives involving that function
will have to be computed using finite difference instead of complex step.  The way to specify
that `fd` should be used for a given derivative is to call `declare_partials`.

In [ ]:
try:
    om.ExecComp.register("unsafe", lambda x: x * x, complex_safe=False)
except NameError:
    pass
p = om.Problem()
comp = p.model.add_subsystem("comp", om.ExecComp("y = 2 * unsafe(x)"))

# because our function is complex unsafe, we must declare that the partials
# with respect to 'x' use 'fd' instead of 'cs'
comp.declare_partials('*', 'x', method='fd')

p.setup()
p.run_model()
J = p.compute_totals(of=['comp.y'], wrt=['comp.x'])
print(J['comp.y', 'comp.x'][0][0])

In [ ]:
assert_near_equal(J['comp.y', 'comp.x'][0][0], 4., 1e-5)

## ExecComp Example: Adding Expressions

You can add additional expressions to an `ExecComp` with the "add_expr" method.

In [ ]:
import numpy as np

class ConfigGroup(om.Group):
    def setup(self):
        excomp = om.ExecComp('y=x',
                             x={'val' : 3.0, 'units' : 'mm'},
                             y={'shape' : (1, ), 'units' : 'cm'})

        self.add_subsystem('excomp', excomp, promotes=['*'])

    def configure(self):
        self.excomp.add_expr('z = 2.9*x',
                             z={'shape' : (1, ), 'units' : 's'})

p = om.Problem()
p.model.add_subsystem('sub', ConfigGroup(), promotes=['*'])
p.setup()
p.run_model()

print(p.get_val('z'))
print(p.get_val('y'))

In [ ]:
assert_almost_equal(p.get_val('z'), 8.7, 1e-8)
assert_almost_equal(p.get_val('y'), 3.0, 1e-8)

## ExecComp Example: Constants

You can define variables in `ExecComp` to be constants.
Here is a simple model in which no constants are used. Notice the names of the inputs that are printed in the two models.

In [ ]:
prob = om.Problem()
C1 = prob.model.add_subsystem('C1', om.ExecComp('x = a + b'))
prob.setup()
prob.set_solver_print(level=0)
prob.run_model()
print(list(C1._inputs._names))

In [ ]:
assert('a' in C1._inputs)
assert('b' in C1._inputs)
assert('x' in C1._outputs)
assert_near_equal(C1._outputs['x'], 2.0, 0.00001)

In [ ]:
prob = om.Problem()
C1 = prob.model.add_subsystem('C1', om.ExecComp('x = a + b', a={'val': 6, 'constant':True}))
prob.setup()
prob.set_solver_print(level=0)
prob.run_model()
print(list(C1._inputs._names))

In [ ]:
assert(not ('a' in C1._inputs))
assert('b' in C1._inputs)
assert('x' in C1._outputs)
assert_near_equal(C1._outputs['x'], 7.0, 0.00001)